# Продвинутый Python, Семинар 3

**Лектор:** Петров Тимур

**Семинаристы:** Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся, Садуллаев Музаффар

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

### Тестирование и логгирование

Для начала поговорим про тестирование. Для чего нужно тестирование?

Пирамида тестирования - один из способов обеспечения качества ПО, визуализация, которая помогает группировать тесты по типу их назначения. Так же, позволяет согласовать правила написания тестов, разделения их на типы, обозначить основной фокус тестирования в каждой из групп.

![image.png](https://habrastorage.org/getpro/habr/post_images/900/55f/47a/90055f47ac1721f8639c950fb3595af6.png)

### Категории тестов

* Юнит-тесты
* Системные тесты (они же интеграционные)
* e2e (end-to-end) тестирование

## Unit Testing

* находятся на самой нижней частей пирамиды и являются основой всех тестов
* тесты проверяют отдельные компоненты (классы, функции и тд) наименьшие единицы программного обеспечения
* главная задача: выявить мелкие баги и ускорить процесс разработки и погружения новых участников проекта в уже написанный код

**Пример:** Тестирование логики функции фильтрации входящего запроса

## Integration Testing

* средний уровень в пирамиде
* тесты помогают проверять как различные системы взаимодействуют между собой
* при написании их тестируются случаи зависящие косвенно от разработчика (к примеру внешняя система не отвечает и мы не должны этот случай обрабатывать!)

### Что может представлять из себя внешняя система?

Все системы которые не могут поддерживаться разработчиками.

* внешнее API
* база данных


**Пример:** Тестирование методов или функций которые взаимодействуют с базой данных

Юнит-тесты работают по принципу «тестирование компонента в изоляции», в то время как
интеграционные тесты проверяют работу различных компонентов вкупе, зачастую интеграционные тесты
отвечают функциональным требованиям непосредственно.

## E2E Testing

* находятся на вершине пирамиды и являются самыми дорогими по времени выполнения
* тесты эмулируют взаимодействие пользователя с системой через UI или интефейс API (swagger) проверяя все компоненты работыют как ожидается
* проводятся реже т.к. дорогие и требует много времени физических и нефизических ресурсов

**Пример:** тестирование пользовательского сценария в интернет магазине от авторизации до оплаты за покупку товара попутно повторяя поведение обычного пользователя.

### Задание на сегодня: покрыть CRUD тестами

Перед вами очень простое CRUD* приложение

CRUD (сокр. от англ. create, read, update, delete — «создать, прочесть, обновить, удалить») — акроним, обозначающий четыре базовые функции, используемые при работе с персистентными хранилищами данных:

* создание
* чтение
* редактирование
* удаление

В коде ниже мы храним всех пользователей в словаре. Ключи мы генерируем сами. От пользователя ждем только обязательное поле `special_id` которое будет у нас будет обязательным при создании клиента. Конечно можно было придумать и внешнюю проверку отдельной функцией чтобы это как то проверять, но давайте это опустим.

In [ ]:
%%writefile simpledb.py

import typing as tp

class SimpleDB:
    def __init__(self):
        self.db = {}

    def create(self, record_id, special_id, name, surname, phone_number) -> str:
        if record_id in self.db:
            raise ValueError(f"Record with ID {record_id} already exists.")
        self.db[record_id] = {
            "special_id": special_id,
            "name": name,
            "surname": surname,
            "phone_number": phone_number
        }
        return f"Record added with ID {record_id}: {self.db[record_id]}"

    def read_all(self) -> dict[str, tp.Any]:
        if not self.db:
            return "Database is empty."
        return self.db

    def read(self, record_id: int) -> str | tp.Any:
        return self.db.get(record_id)

    def update(self, record_id: int, special_id: int, name=None, surname=None, phone_number=None) -> str:
        if record_id not in self.db:
            return f"Record with ID {record_id} not found."

        self.db[record_id]['special_id'] = special_id

        if name:
            self.db[record_id]['name'] = name
        if surname:
            self.db[record_id]['surname'] = surname
        if phone_number:
            self.db[record_id]['phone_number'] = phone_number
        return f"Record with ID {record_id} updated: {self.db[record_id]}"

    def delete(self, record_id: int) -> str:
        if record_id not in self.db:
            return f"Record with ID {record_id} not found."
        deleted_record = self.db.pop(record_id)
        return f"Record deleted with ID {record_id}: {deleted_record}"


Overwriting simpledb.py


In [ ]:
%%writefile user_service.py

import typing as tp

from simpledb import SimpleDB

class UserService:
    def __init__(self, name: str):
        self._name = name
        self.__db = None

    @property
    def _db(self) -> SimpleDB:
        if self.__db is None:
            self.__db = SimpleDB()
        return self.__db

    def _data_validation(self, **kwargs) -> None:
        if 'special_id' not in kwargs or not isinstance(kwargs['special_id'], int):
            raise ValueError("The field 'special_id' is required and must be an integer.")
        for field in ['name', 'surname', 'phone_number']:
            if field not in kwargs or not kwargs[field]:
                raise ValueError(f"The field '{field}' is required.")

    def create_new_user(self, **kwargs) -> str | tp.Any:
        import random
        self._data_validation(**kwargs)

        new_id = random.randint(0, int(1e9))
        while new_id in self._db.db:
            new_id = random.randint(0, int(1e9))

        self._db.create(new_id, kwargs['special_id'], kwargs['name'], kwargs['surname'], kwargs['phone_number'])
        return f"Successfully created new user with ID {new_id}"

    def get_all_users(self) -> str:
        users = self._db.read_all()
        if isinstance(users, str):
            return users
        return f"All users: {users}"

    def get_user_by_id(self, user_id):
        user = self._db.read(user_id)
        if user is None:
            raise ValueError(f"User with ID {user_id} not found.")
        return user

    def update_user(self, user_id, **kwargs):
        user_data = self.get_user_by_id(user_id)
        if user_data is None:
            raise ValueError(f"User with ID {user_id} not found.")

        for key, value in kwargs.items():
            if value is not None:
                user_data[key] = value

        updated_user = self._db.update(
            user_id,
            user_data['special_id'],
            user_data.get('name'),
            user_data.get('surname'),
            user_data.get('phone_number')
        )
        return updated_user

    def delete_user(self, user_id) -> str:
        deleted_user = self._db.delete(user_id)
        return deleted_user

Overwriting user_service.py


In [ ]:
from simpledb import SimpleDB
from user_service import UserService

TypeError: unsupported operand type(s) for |: 'type' and '_SpecialForm'

Убедимся, что у нас нулевое покрытие тестами. Как проверить и что это такое?

Cуществует такой модуль как covarage.py в котором она индексирует все задетые и проверенные случаи в тестах приложения.

Теперь давайте его покроем тестами. В первую очередь в логике мы прописывали что при создании экземляра класса мы не создаем инстанс базы данных, а только при первом использовании. Давайте убедимся в этом:

In [ ]:
# настроим все необходимое чтобы гонять тесты в пайбуке

import pytest
import ipytest
import coverage
ipytest.autoconfig()
__file__ = "Deep_Seminar_03.ipynb"

Чтобы проверить что объект невызывался достаточно сделать возвращаемое значение пустым и проверить количество вызовов:

In [ ]:
%%ipytest -q

import pytest
from mock import patch, PropertyMock

from user_service import UserService

def test_db_not_initialized_until_accessed():
    with patch.object(UserService, '_db', new_callable=PropertyMock) as mock_db:
        service = UserService(name="test_service")

    assert mock_db.call_count == 0


.                                                                                            [100%]


Мы тут подменили свойство в классе `UserService` с помощью `PropertyMock`. Это позволило нам проверять свойство `_db` сколько раз оно вызывается, и проверить, был ли объект базы данных инициализирован. По тесту мы убедились что свойство не вызывалось все как и ожидалось.

А что если захотим проверить что оно у нас сначало не вызывалось, а после вызова метода где он требуется вызовется? Давайте допишем тестовый сценарий:
Красиво это будет через паратмеризацию:

In [ ]:
%%ipytest -q

from user_service import UserService

def test_db_initialization():
    with patch.object(UserService, '_db', new_callable=PropertyMock) as mock_db:
        service = UserService(name="test_service")

        assert mock_db.call_count == 0

        service.get_all_users()

        assert mock_db.call_count == 1

.                                                                                            [100%]


Вроде ок. Даже можно сказать что читаемо. А если это сделать при помощи параметризации?

In [ ]:
%%ipytest -q

from user_service import UserService

def test_db_initialization(is_called):
    raise NotImplemented("paper please")

..                                                                                           [100%]


Давайте теперь покроем каждый метод класса соответсвующими юнит тестами c подменой нашей базой данных, чтобы у нас получились полноценные юнит тесты!

Для того чтобы покрыть это все юнитами, нам нужна фикстура базы данных которая будет вести себя также как и реальная.

In [ ]:
%%ipytest -q

import pytest
from mock import patch, Mock, MagicMock

from simpledb import SimpleDB
from user_service import UserService

@pytest.fixture
def mock_db():
    mock_db = Mock()
    mock_db.db = {}
    return mock_db

@pytest.fixture
def user_service(mock_db):
    service = UserService(name='TestService')
    service._UserService__db = mock_db
    return service

Здесь мы создали класс Mock который имитирует поведение нашей базы данных. Также создали моканный сервис. Конкретный интересный момент здесь: `service._UserService__db = mock_db`. \
Мы определили приватное поле db в cвоей реализации и не просто так. Когда определяются приватные поля, то у нас срабатывает механизм, называемый "Name Mangling". \
Оно используется для предотвращения конфликтов имён в подклассах и для того, чтобы сделать атрибуты "приватными", то есть не легко доступными из вне класса.

То есть если:

```python

class A:
    def __init__(self):
        self.__a = None

```

Атрибут с именем `A.__a` внутри становится `_A__a`. То есть если извне мы захотим его заиспользовать, то можно его вызывать как `some_object._A__a`. Сделано это для избежания конфликтов и инкапсуляции.



Теперь мы хотим с вами создать тестового клиента. Но по реализации у нас рандом генерит id клиента. Что же делать? \
Чтобы сделать тест предсказуемым, мы можем заменить функцию `random.randint` на фиктивную функцию, которая всегда возвращает одно и то же значение. В этом помогать будет `patch`. \
В аргументах мы указываем точный путь к функции, которую хотим заменить и ее возвращаемое значение.

In [ ]:
%%ipytest -q

from mock import patch

def test_create_new_user(user_service, mock_db):
    user_data = {
        'special_id': 123,
        'name': 'Иван',
        'surname': 'Иванов',
        'phone_number': '+71234567890'
    }

    with patch('random.randint', return_value=1):
        result = user_service.create_new_user(**user_data)

    mock_db.create.assert_called_with(1, 123, 'Иван', 'Иванов', '+71234567890')
    assert result == 'Successfully created new user with ID 1'

.                                                                                            [100%]


**Задание:** Написать тест получения всех пользователей, имитируя поход в базу

In [ ]:
%%ipytest -q

users = [
        {'special_id': 123, 'name': 'Иван', 'surname': 'Иванов', 'phone_number': '+71234567890'},
        {'special_id': 456, 'name': 'Мария', 'surname': 'Петрова', 'phone_number': '+79876543210'}
    ]

def test_get_all_users(user_service, mock_db):
    raise NotImplemented("paper please")

.                                                                                            [100%]


**Задание:** Написать тест доставания людей по ключу, имитируя поход в базу

In [ ]:
%%ipytest -q

user_id = 1
user_data = {'special_id': 123, 'name': 'Иван', 'surname': 'Иванов', 'phone_number': '+71234567890'}

def test_get_user_by_id(user_service, mock_db):
    raise NotImplemented("paper please")

.                                                                                            [100%]


**Задание:** Написать тест где мы ожидаем ошибку что пользователь не найден

In [ ]:
%%ipytest -q

def test_get_user_by_id_not_found(user_service, mock_db):
    raise NotImplemented("paper please")

.                                                                                            [100%]


**Задание:** Написать тест на обновление пользователя которого нет (корректно обрабатывая ошибку!)

In [ ]:
%%ipytest -q

def _message(user_id):
    return f"User with ID {user_id} not found"

def test_update_user_not_found(user_service, mock_db):
    return NotImplemented("paper please")

.                                                                                            [100%]


**Задание:** Написать тест на удаление пользователя

In [ ]:
%%ipytest -q


def _message_2(user_id):
    return f"Record deleted with ID {user_id}: <user data>"

def test_delete_user(user_service, mock_db):
    return NotImplemented("paper please")

.                                                                                            [100%]


Если мы ожидаем ошибку, то нужно прописывать `pytest.raises(<класс ошибки>)`. Это говорит pytest'у что мы ждем ошибку, и падать если так таковая не случилась, например:

In [ ]:
%%ipytest -q

from contextlib import nullcontext

def test_create_new_user_missing_fields_(user_service):
    user_data = {
        'special_id': 123,
        'name': 'Иван',
        'phone_number': '+71234567890'
    }
    user_service.create_new_user(**user_data)


F                                                                                            [100%]
============================================= FAILURES =============================================
_______________________________ test_create_new_user_missing_fields_ _______________________________

user_service = <user_service.UserService object at 0x1164c4700>

    def test_create_new_user_missing_fields_(user_service):
        user_data = {
            'special_id': 123,
            'name': 'Иван',
            'phone_number': '+71234567890'
        }
>       user_service.create_new_user(**user_data)

/var/folders/1k/s7xycc896gd2kbcgp89z21_06b4zqs/T/ipykernel_18810/337764304.py:9: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
user_service.py:24: in create_new_user
    def create_new_user(self, **kwargs) -> str | tp.Any:
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

self =

А теперь:

In [ ]:
%%ipytest -q

def test_create_new_user_missing_fields(user_service):
    user_data = {
        'special_id': 123,
        'name': 'Иван',
        'phone_number': '+71234567890'
    }

    with pytest.raises(ValueError) as exc_info:
        user_service.create_new_user(**user_data)

    assert str(exc_info.value) == "The field 'surname' is required."

.                                                                                            [100%]


In [ ]:
%%ipytest -q

def test_create_new_user_invalid_special_id(user_service):
    user_data = {
        'special_id': 'abc',  # неверный тип
        'name': 'Иван',
        'surname': 'Иванов',
        'phone_number': '+71234567890'
    }

    with pytest.raises(ValueError) as exc_info:
        user_service.create_new_user(**user_data)

    assert str(exc_info.value) == "The field 'special_id' is required and must be an integer."

.                                                                                            [100%]


`side_effect` и `return_value`

Чтобы мок мог вести себя так же, как реальная база данных, нам нужно определить кастомное поведение для его методов. Именно для этого мы используем `side_effect`. Нам нужно сымитировать его поведение без реального вызова объекта, что конкретно данный механизм отлично справляется. \
В свою очередь `return_value` не задает поведение, а возвращает определенное значение что ему прописали.

In [ ]:
%%ipytest -q

def test_update_user(user_service, mock_db):
    user_id = 1
    existing_user_data = {'special_id': 123, 'name': 'Иван', 'surname': 'Иванов', 'phone_number': '+71234567890'}

    mock_db.db[user_id] = existing_user_data.copy()

    def read_side_effect(record_id):
        return mock_db.db.get(record_id)
    mock_db.read.side_effect = read_side_effect

    def update_side_effect(record_id, special_id, name, surname, phone_number):
        if record_id not in mock_db.db:
            return f"Record with ID {record_id} not found."

        user = mock_db.db[record_id]
        user['special_id'] = special_id
        if name is not None:
            user['name'] = name
        if surname is not None:
            user['surname'] = surname
        if phone_number is not None:
            user['phone_number'] = phone_number

        return f"Record with ID {record_id} updated: {user}"

    mock_db.update.side_effect = update_side_effect

    update_data = {'name': 'Иван1', 'phone_number': '+79991234567'}

    result = user_service.update_user(user_id, **update_data)

    expected_data = existing_user_data.copy()
    expected_data.update({k: v for k, v in update_data.items() if v is not None})

    assert mock_db.db[user_id] == expected_data
    assert result == f"Record with ID {user_id} updated: {expected_data}"

.                                                                                            [100%]


### Интеграционные тесты

Как только мы с вами реализовали unit-тесты, давайте реализовать интеграционные. Отличаются они от юнитов тем, что мы ходим в реальный сервис (в нашем случае это наша база данных) непосредственно обращаясь к ней

!!! Важно \
Если вы обращаетесь к реальной базе данных для этого вы должны подготовить для нее тестовую среду (саму тестовую базу) и настроить все чтобы она не записывала и не трогала продовую

В рамках тестирования необходимо иметь контроль даже над тем чем мы не можем. К примеру модуль `random`. Можно много чего от него ожидать. Но, давайте его пропатчим чтобы он работал так как нам надо ибо это логика не очень та и важна в рамках тестирования системы. При вызове `random.randint` мы будем возвращать число `1`

In [ ]:
%%ipytest -q

@pytest.fixture(autouse=True)
def fixed_random_id(monkeypatch):
    import random

    original_randint = random.randint

    def fixed_randint(a, b):
        return 1

    monkeypatch.setattr(random, 'randint', fixed_randint)

    yield
    monkeypatch.setattr(random, 'randint', original_randint)

Теперь напишем тест, что при каждом вызове `random.randint` у нас возвращается число 1.

In [ ]:
%%ipytest -q

@pytest.fixture
def user_service():
    service = UserService(name='TestService')
    return service

def test_create_new_user(user_service):
    user_data = {
        'special_id': 123,
        'name': 'Иван',
        'surname': 'Иванов',
        'phone_number': '+71234567890'
    }

    result = user_service.create_new_user(**user_data)

    assert result == 'Successfully created new user with ID 1'

    user = user_service.get_user_by_id(1)
    assert user == user_data

.                                                                                            [100%]


In [ ]:
%%ipytest -q
import random

def test_get_all_users(user_service):
    users = [
        {'special_id': 123, 'name': 'Иван', 'surname': 'Иванов', 'phone_number': '+71234567890'},
        {'special_id': 456, 'name': 'Мария', 'surname': 'Петрова', 'phone_number': '+79876543210'}
    ]

    for i, user_data in enumerate(users, start=1):
        with pytest.MonkeyPatch().context() as m:
            m.setattr(random, 'randint', lambda a, b, i=i: i)
            user_service.create_new_user(**user_data)

    result = user_service.get_all_users()

    expected_output = f"All users: {user_service._db.db}"
    assert result == expected_output


.                                                                                            [100%]


**Задание** Протестировать метод `get_user_by_id` обращаясь к реальной базе

In [ ]:
%%ipytest -q

def test_get_user_by_id(user_service):
    user_data = {
        'special_id': 123,
        'name': 'Иван',
        'surname': 'Иванов',
        'phone_number': '+71234567890'
    }
    user_service.create_new_user(**user_data)

    user = user_service.get_user_by_id(1)
    assert user == user_data

.                                                                                            [100%]


Давайте теперь проверим без заготовок как у нас будет вести тест при несуществующем пользователе:

In [ ]:
%%ipytest -q

def test_get_user_by_id_not_found(user_service):
    with pytest.raises(ValueError) as exc_info:
        user_service.get_user_by_id(999)

    assert str(exc_info.value) == "User with ID 999 not found."

.                                                                                            [100%]


Как видим поведение одно и тоже!

**Задание** Необходимо протестировать метод обновление пользователя на реальной базе

In [ ]:
%%ipytest -q

user_data = {
        'special_id': 123,
        'name': 'Иван',
        'surname': 'Иванов',
        'phone_number': '+71234567890'
    }

def test_update_user(user_service):

   raise NotImplemented("paper please")

.                                                                                            [100%]


Давайте в тесте не просто создадим и удалим, а еще запросим уже удаленного клиента. \
**Notice**: в реальных тестах так делать нельзя.
Главная особенность теста: тест должен быть говорящим! \
Дополнительная логика в нем излишна, и перегружать функционал теста:
* бесполезно
* он может ввести в заблуждение

In [ ]:
%%ipytest -q

def test_delete_user(user_service):
    user_data = {
        'special_id': 123,
        'name': 'Иван',
        'surname': 'Иванов',
        'phone_number': '+71234567890'
    }
    user_service.create_new_user(**user_data)

    result = user_service.delete_user(1)
    assert result == f"Record deleted with ID 1: {user_data}"

    # Проверяем, что пользователя больше нет в базе данных
    with pytest.raises(ValueError) as exc_info:
        user_service.get_user_by_id(1)

    assert str(exc_info.value) == "User with ID 1 not found."

.                                                                                            [100%]


**Задание** Протестировать случай при создании клиента отсутвие `required` поля

In [ ]:
%%ipytest -q

user_data = {
        'special_id': 123,
        'name': 'Иван',
        # Отсутствует 'surname'
        'phone_number': '+71234567890'
    }


def test_create_new_user_missing_fields(user_service):
    raise NotImplemented("paper please")


.                                                                                            [100%]


**Задание** Протестировать случай при обновление несуществующего клиента

In [ ]:
%%ipytest -q

def test_update_user_not_found(user_service):

    raise NotImplemented("paper please")

.                                                                                            [100%]


И в последнем тесте мы проверяем что наш "уникальный ключ" не целочисленный:

In [ ]:
%%ipytest -q

def test_create_new_user_invalid_special_id(user_service):
    user_data = {
        'special_id': 'abc',  # Некорректный тип
        'name': 'Иван',
        'surname': 'Иванов',
        'phone_number': '+71234567890'
    }

    with pytest.raises(ValueError) as exc_info:
        user_service.create_new_user(**user_data)

    assert str(exc_info.value) == "The field 'special_id' is required and must be an integer."

.                                                                                            [100%]


### Тэги в тестах

Для человекочитаемости различных параметров мы можем присваивать ему имя или id. К примеру:

In [ ]:
%%ipytest -q

import pytest


import pytest

@pytest.mark.parametrize(
    "a, b, c",
    [
        pytest.param(1, 2, 3), id="Мои первые числа!!!",
        pytest.param(4, 5, 9, id="когда нечего придумать...")
    ]
)
def test_a_plus_b(a, b, c):
    assert a + b == c


..                                                                                           [100%]


Главная фишка такая, берете свой набор параметров и оборачиваете их `pytest.param` и указываете ему `id=""` вуаля и готово!

### Логгирование

Вернемся к написанным ранее классам. Давайте добавим на каждый событие в методах класса лог

In [ ]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

Что мы здесь установили?
* уровень логгирования сообщения уровнем INFO и выше будут записываться
* определяем формат сообщений лога (кастомно)
* определяем форматы записи даты (есть стандартные даты, но тоже мы указываем кастомно)

Теперь можем спокойно добавлять в наши классы!

In [ ]:
class SimpleDB:
    def __init__(self):
        self.db = {}
        self.logger = logging.getLogger(self.__class__.__name__)

    def create(self, record_id, special_id, name, surname, phone_number):
        if record_id in self.db:
            self.logger.error(f"Attempt to create a record with existing ID {record_id}")
            raise ValueError(f"Record with ID {record_id} already exists.")
        self.db[record_id] = {
            "special_id": special_id,
            "name": name,
            "surname": surname,
            "phone_number": phone_number
        }
        self.logger.info(f"Record added with ID {record_id}: {self.db[record_id]}")
        return f"Record added with ID {record_id}: {self.db[record_id]}"

    def read_all(self):
        if not self.db:
            self.logger.warning("Read all called, but database is empty.")
            return "Database is empty."
        self.logger.info("Read all records from the database.")
        return self.db

    def read(self, record_id: int):
        record = self.db.get(record_id)
        if record:
            self.logger.info(f"Record read with ID {record_id}: {record}")
        else:
            self.logger.warning(f"Record with ID {record_id} not found.")
        return record

    def update(self, record_id: int, special_id: int, name=None, surname=None, phone_number=None):
        if record_id not in self.db:
            self.logger.error(f"Attempt to update non-existent record with ID {record_id}")
            return f"Record with ID {record_id} not found."

        self.db[record_id]['special_id'] = special_id

        if name:
            self.db[record_id]['name'] = name
        if surname:
            self.db[record_id]['surname'] = surname
        if phone_number:
            self.db[record_id]['phone_number'] = phone_number

        self.logger.info(f"Record updated with ID {record_id}: {self.db[record_id]}")
        return f"Record with ID {record_id} updated: {self.db[record_id]}"

    def delete(self, record_id: int):
        if record_id not in self.db:
            self.logger.error(f"Attempt to delete non-existent record with ID {record_id}")
            return f"Record with ID {record_id} not found."
        deleted_record = self.db.pop(record_id)
        self.logger.info(f"Record deleted with ID {record_id}: {deleted_record}")
        return f"Record deleted with ID {record_id}: {deleted_record}"

In [ ]:
class UserService:
    def __init__(self, name: str):
        self._name = name
        self.__db = None
        self.logger = logging.getLogger(self.__class__.__name__)

    @property
    def _db(self):
        if self.__db is None:
            self.__db = SimpleDB()
            self.logger.debug("Initialized SimpleDB instance.")
        return self.__db

    def _data_validation(self, **kwargs):
        self.logger.debug(f"Validating data: {kwargs}")
        if 'special_id' not in kwargs or not isinstance(kwargs['special_id'], int):
            self.logger.error("Validation error: 'special_id' is missing or not an integer.")
            raise ValueError("The field 'special_id' is required and must be an integer.")
        for field in ['name', 'surname', 'phone_number']:
            if field not in kwargs or not kwargs[field]:
                self.logger.error(f"Validation error: The field '{field}' is missing or empty.")
                raise ValueError(f"The field '{field}' is required.")

    def create_new_user(self, **kwargs):
        import random
        self._data_validation(**kwargs)

        new_id = random.randint(0, int(1e9))
        while new_id in self._db.db:
            self.logger.debug(f"Generated duplicate ID {new_id}, generating a new one.")
            new_id = random.randint(0, int(1e9))

        self.logger.info(f"Creating new user with ID {new_id}")
        self._db.create(new_id, kwargs['special_id'], kwargs['name'], kwargs['surname'], kwargs['phone_number'])
        return f"Successfully created new user with ID {new_id}"

    def get_all_users(self):
        self.logger.info("Fetching all users.")
        users = self._db.read_all()
        if isinstance(users, str):
            return users
        return f"All users: {users}"

    def get_user_by_id(self, user_id):
        self.logger.info(f"Fetching user with ID {user_id}")
        user = self._db.read(user_id)
        if user is None:
            self.logger.error(f"User with ID {user_id} not found.")
            raise ValueError(f"User with ID {user_id} not found.")
        return user

    def update_user(self, user_id, **kwargs):
        self.logger.info(f"Updating user with ID {user_id}")
        user_data = self.get_user_by_id(user_id)
        if user_data is None:
            self.logger.error(f"User with ID {user_id} not found for update.")
            raise ValueError(f"User with ID {user_id} not found.")

        for key, value in kwargs.items():
            if value is not None:
                self.logger.debug(f"Updating field '{key}' with value '{value}'")
                user_data[key] = value

        updated_user = self._db.update(
            user_id,
            user_data['special_id'],
            user_data.get('name'),
            user_data.get('surname'),
            user_data.get('phone_number')
        )
        return updated_user

    def delete_user(self, user_id):
        self.logger.info(f"Deleting user with ID {user_id}")
        deleted_user = self._db.delete(user_id)
        return deleted_user

Конкретно тут интересует `self.logger = logging.getLogger(self.__class__.__name__)` \
Здесь мы создаем класс логгера с именем класса где он находится.


In [ ]:
if __name__ == '__main__':
    service = UserService('MainService')

    try:
        service.create_new_user(
            special_id=123,
            name='Иван',
            surname='Иванов',
            phone_number='+71234567890'
        )
    except ValueError as e:
        logging.error(e)

    users = service.get_all_users()
    print(users)

    try:
        service.update_user(1, name='Иван Петрович')
    except ValueError as e:
        logging.error(e)

    try:
        service.delete_user(1)
    except ValueError as e:
        logging.error(e)

ERROR:UserService:User with ID 1 not found.
ERROR:root:User with ID 1 not found.
ERROR:SimpleDB:Attempt to delete non-existent record with ID 1


All users: {691336785: {'special_id': 123, 'name': 'Иван', 'surname': 'Иванов', 'phone_number': '+71234567890'}}
